#### **Deep & Reinforcement Learning**

#### **Observações**

##### Redes Adversárias Generativas (GANs)

Vamos aprender a aplicar as Redes Adversárias Generativas (GANs)? Vamos utilizar a clássica base de dados MNIST. Esse dataset é disponibilizado pelo Keras e consistem em imagens de treinamento e imagens de teste para classificar dígitos escritos a mão que variam entre 0 a 9.

O que são as redes adversárias gerativas? 🤔

As redes Adversárias Generativas possuem um grande potencial pois são capazes de gerar novos dados a partir de um conjunto de dados treinados. Podemos treinar essa rede para criar, por exemplo, novas imagens, músicas, falas, prosas, tratar resoluções de imagens e vídeos e muito mais. Uma das suas utilidades também pode ser criar novas imagens a partir de um conjunto de dados real para criar mais amostras de dados ao treinar uma rede neural convolucional.

Você já viu aquelas imagens fakes? Pois é, as redes GANs tem tanto o poder de criar dados fakes quanto validar a veracidade de dados. A rede GANs também é muito utilizada para recriar partes de imagens, por exemplo, se temos uma imagem de um cachorro pela metade, a rede tem o incrível poder de recriar. Falando ainda do uso da rede em imagens, a Pixar que é uma grande empresa que trabalha com animações utiliza muito as redes GANs para aumento de resolução de imagens. Para quem não conhece as redes Adversárias Generativas até parece mágica, não é mesmo?

Você aprendeu na aula de redes não supervisionadas que algoritmos discriminativos tentam criar classes ou grupos a partir de dados de entrada, ou seja, mapeiam recursos para criar rótulos utilizando correlação. Os algoritmos generativos fazem justamente o oposto, eles tentam prever os recursos (dados) com um determinado rótulo. Por exemplo, dado que um e-mail é classificado com spam, qual é a probabilidade de palavras que formam esse e-mail spam? Os algoritmos discriminativos se preocupam com a correlação entre x e y, modelos generativos se preocupam em “como você irá obter x”.

##### **Como criar ambientes para evitar conflito:**
https://github.com/RicardViana/fiap-data-viz-and-production-models/blob/main/Roteiro%20para%20cria%C3%A7%C3%A3o%20de%20ambiente.pdf

#### **Conteúdo - Bases e Notebook da aula**

Github:  
https://github.com/FIAP/Pos_Tech_DTAT/tree/DeepLearning

ou

https://github.com/FIAP/Pos_Tech_DTAT/tree/main


#### **Importação de bibliotecas**

In [ ]:
#Importar biblioteca completa
import os
import glob     
import time
import imageio  
import PIL      
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

#Importar algo especifico de uma biblioteca
from pathlib import Path
from IPython import display # Para limpar a saída do notebook e atualizar a imagem "ao vivo"
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Reshape, Conv2DTranspose, Conv2D, LeakyReLU, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tqdm import tqdm 

print(f"TensorFlow versão: {tf.__version__}")
print("Ambiente configurado para criar, treinar e gerar GIFs da GAN!")

#### **Funções (def)**

In [ ]:

'''

Basicamente as GANs são compostas por duas redes, a geradora e a discriminante. São chamadas de adversárias porque essas duas redes tentam “jogar” uma com a outra o tempo todo.
A rede geradora tenta enganar a rede discriminante gerando dados fakes semelhantes aos dados reais.

O objetivo da rede discriminate aqui é reconhecer que os dígitos gerados a mão são o mais próximos possíveis dos verdadeiros números. 
A rede geradora tenta criar novas imagens fakes com a esperança de torná-las autênticas também (mesmo sendo falsas). 
O funcionamento da rede consiste nas seguintes etapas: O gerador considera números aleatórios e retornam uma imagem (ou seja, cria uma imagem fake); 
Essa imagem fake gerada pelo gerador é inserida no discriminador ao lado do fluxo de imagens verdadeiras geradas; o discriminador obtém imagens reais e falsas retornando 
a probabilidade realizando previsões de imagens falsas geradas pela rede generativa.

'''

# criando o nosso modelo gerador

def make_generator_model():
  model = tf.keras.Sequential()
  model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Reshape((7,7,256)))
  assert model.output_shape == (None, 7, 7, 256) # None é o batch size

  model.add(layers.Conv2DTranspose(128,(5, 5), strides=(1,1), padding='same', use_bias=False))
  assert model.output_shape == (None, 7, 7, 128)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(64,(5, 5), strides=(2,2), padding='same', use_bias=False))
  assert model.output_shape == (None, 14, 14, 64)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(1,(5, 5), strides=(2,2), padding='same', use_bias=False, activation='tanh'))
  assert model.output_shape == (None, 28, 28, 1)

  return model

In [ ]:
def make_discriminator_model():
  model = tf.keras.Sequential()
  model.add(layers.Conv2D(64, (5, 5), strides=(2,2), padding='same', input_shape=[28,28,1]))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(128, (5, 5), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  return model

In [ ]:
# função loss para o discriminador

def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output) # dado real # 1 verdadeiro
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output) # dado fake # 0 falso
  total_loss = real_loss + fake_loss # faz a comparação
  
  return total_loss

In [ ]:
# avaliar fake gerado
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output ), fake_output) # dado fake apenas

In [ ]:
# criando uma função para mostrar a evolução dos passos de treinamento

# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
# função para definir as imagens

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

#### **Aula 6 - Gans**

In [ ]:
# Subindo a base de dados
(train_images, train_labels), (_,_) = tf.keras.datasets.mnist.load_data()

In [ ]:
# Ajustar as imagens
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') # definindo o tamaho da imagm (28x28) com 1 canal de cinza
train_images = (train_images - 127.5) / 127.5 # normalizando a imagem para -1, 1

In [ ]:
# Set do tamanho do conjuto de dados e a qtd de pixels
BUFFER_SIZE = 60000 # tamanho do conjunto de dados
BATCH_SIZE = 256 # quantidade de pixels que varia

In [ ]:
# criando a base de treinamento
# from_tensor_slices: imagens dentro de um tensor
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE) #conjunto de dados + batch para normalizar os pixels

In [ ]:
'''

Agora utilizando a função geradora de imagens, vamos ver o que ela pode criar de forma aleatória.
Criando a primeira imagem nunca treinada antes:

'''

generator = make_generator_model()

noise = tf.random.normal([1, 100])
generator_image = generator(noise, training=False)

plt.imshow(generator_image[0, :, :, 0], cmap='gray')


In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generator_image)
print(decision)

In [ ]:
# Configurando a função de custo (loss function)

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False) # crio uma função loss

In [ ]:
# Definindo o otimizador da função de custo Adam
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# Para salvar o modelo
checkpoint_dir = './training_checkpoints' #diretório para salvar pontos específicos de rede
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                               discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)

In [ ]:
# Definir as épocas de processamento:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Realiozar o treino:
train(train_dataset, EPOCHS)